In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:

def accuracy(predictions, labels):
      return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

### L2 

In [5]:
def initiate(num_input, num_output):
    w = tf.Variable(
    tf.truncated_normal([num_input, num_output]))
    b = tf.Variable(tf.zeros([num_output]))
    
    return w, b

def model(X, w1, b1, w2, b2): 
    logits1 = tf.matmul(X, w1) + b1
    relu1 = tf.nn.relu(logits1)
    logits2 = tf.matmul(relu1, w2) + b2
    return logits2


In [ ]:
batch_size = 128  ## subset size

graph = tf.Graph()
with graph.as_default():

    # Placeholder that will fed the training data
    # at run time with a training minibatch.
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
    # initiate weights
    weight1, biases1 = initiate(784,1024)
    weight2, biases2 = initiate(1024,10)
    
    # computation
    prediction = model(tf_train_dataset, weight1, biases1, weight2, biases2)

    # Loss + L2 Loss
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=prediction)) 
    + 0.01*(tf.nn.l2_loss(weight1) + tf.nn.l2_loss(weight2) 
            + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(biases2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(
        model(tf_train_dataset, weight1, biases1, weight2, biases2))
    valid_prediction = tf.nn.softmax(
        model(tf_valid_dataset, weight1, biases1, weight2, biases2))
    test_prediction = tf.nn.softmax(
        model(tf_test_dataset, weight1, biases1, weight2, biases2))

In [ ]:
num_steps = 5000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = np.random.randint(0,train_dataset.shape[0]/batch_size)*batch_size
        
        # Random Subsetting
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, prediction], feed_dict=feed_dict)
        # since the feeding data is different now. So we need tell the model
        # the data we want to feed by using feed_dict.
    
        if (step % 1000 == 0):
            print("Offset: %d" % offset)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

### L2 + Dropout + Decay Learning Rate

In [161]:
def initiate(num_input, num_output):
    w = tf.Variable(
    tf.truncated_normal([num_input, num_output]))
    b = tf.Variable(tf.zeros([num_output]))
    
    return w, b

def model(X, w1, b1, w2, b2): 
    logits1 = tf.matmul(X, w1) + b1
    relu1 = tf.nn.relu(logits1)
    logits2 = tf.matmul(relu1, w2) + b2
    return logits2


def model_dropout(X, w1, b1, w2, b2): 
    logits1 = tf.matmul(X, w1) + b1
    relu1 = tf.nn.relu(logits1)
    
    ## Adding dropout
    drop_out = tf.nn.dropout(relu1, 0.5)
    
    logits2 = tf.matmul(drop_out, w2) + b2
    
    return logits2

In [164]:
batch_size = 128  ## subset size

graph = tf.Graph()
with graph.as_default():

    # Placeholder that will fed the training data
    # at run time with a training minibatch.
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
    # initiate weights
    weight1, biases1 = initiate(784,1024)
    weight2, biases2 = initiate(1024,10)
    
    # computation
    prediction = model(tf_train_dataset, weight1, biases1, weight2, biases2)

    # Loss + L2 Loss
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=prediction)) 
    + 0.001*(tf.nn.l2_loss(weight1) + tf.nn.l2_loss(biases1)  
            + tf.nn.l2_loss(weight2) + tf.nn.l2_loss(biases2))
    
    # Decay Learning Rate
    global_step = tf.Variable(0, name= 'global_step', trainable = False)
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.96, staircase=True)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step) 
        
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(
        model(tf_train_dataset, weight1, biases1, weight2, biases2))
    valid_prediction = tf.nn.softmax(
        model(tf_valid_dataset, weight1, biases1, weight2, biases2))
    test_prediction = tf.nn.softmax(
        model(tf_test_dataset, weight1, biases1, weight2, biases2))

In [166]:
num_steps = 10

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Random Subsetting
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
                     
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, prediction], feed_dict=feed_dict)
        # since the feeding data is different now. So we need tell the model
        # the data we want to feed by using feed_dict.
        
        
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Learning Rate: ", learning_rate.eval())
        if (step % 5000 == 0):
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))            

Initialized
Minibatch loss at step 0: 437.366058
Minibatch accuracy: 6.2%
Validation accuracy: 16.9%
Learning Rate:  0.05
Test accuracy: 18.1%
Test accuracy: 65.5%
